<a href="https://colab.research.google.com/github/chris-creditdesign/nih-grant-terminations/blob/main/total_funding_remaining.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# Setup
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from bs4 import BeautifulSoup
import requests

df = pd.read_csv("/content/2024-03-31-NIH Grant Terminations_ Auto-Generated - grant_data.csv")

# Drop any rows where the 'cancellation_source' column is not 'HHS reported' or 'Self reported'
df_confirmed = df[df['cancellation_source'].isin(['HHS reported', 'Self reported', 'Self and HHS reported'])]
total_grants_count = len(df_confirmed)
print(f"There are {total_grants_count} rows in the dataframe.")

There are 704 rows in the dataframe.


In [37]:
def get_table_from_nih_website(url, number_of_cols):
  response = requests.get(url)
  soup = BeautifulSoup(response.content, "html.parser")

  table = soup.find("table", {"class": "table-data-tight"})

  # Extract headers
  headers = [th.text.strip() for th in table.find_all("th")]

  # Extract rows
  rows = []
  for tr in table.find_all("tr"):
      row_data = [td.text.strip() for td in tr.find_all("td")]
      if row_data:  # Skip empty rows
          rows.append(row_data)
  # Create the pandas DataFrame
  df = pd.DataFrame(rows, columns=headers[1:number_of_cols])

  return df

In [38]:
section_1_url = "https://www.nih.gov/about-nih/what-we-do/nih-almanac/appropriations-section-1"
section_2_url = "https://www.nih.gov/about-nih/what-we-do/nih-almanac/appropriations-section-2"

In [41]:
section_1_df = get_table_from_nih_website(section_1_url, 15)
section_2_df = get_table_from_nih_website(section_2_url,16)

In [42]:
section_1_df.head()

,NCI11,NHLBI,NIDCR,NIDDK1,NINDS2,NIAID12,NIGMS,NICHD3,NEI,NIEHS4,NIA,NIAMS,NIDCD,NIMH5
0,400,,,,,,,,,,,,,
1,400,,,,,,,,,,,,,
2,570,,,,,,,,,,,,,
3,570,,,,,,,,,,,,,
4,565,,,,,,,,,,,,,


In [52]:
section_2_df.head()

,NIDA,NIAAA,NINR,NHGRI,NIBIB,NIMHD,"NCRR6,7",NCCAM14,NCATS,FIC,NLM,OD8,B&F9,OAR10,Total
0,,,,,,,64,,,,,,,,464
1,,,,,,,64,,,,,,,,464
2,,,,,,,137,,,,,,,,707
3,,,,,,,141,,,,,,,,711
4,,,,,,,135,,,,,,,,700


In [60]:
total = section_2_df["Total"]

In [66]:
total.tail(4)

,Total
83,"41,690,000"
84,"42,940,500"
85,"45,182,990"
86,"47,683,485"


In [108]:
# Times 1000 as total is in 1000s of dollars
total_2020_23 = total.tail(4).str.replace(",", "").astype(int).sum() * 1000
print(f"The sum of the last five values in 'total' is: {total_2020_23}")


The sum of the last five values in 'total' is: 177496975000


In [69]:
# NIH Awards by Location & Organization
# https://report.nih.gov/award/index.cfm?ot=&fy=2024&state=&ic=&fm=&orgid=&distr=&rfa=&om=n&pid=#tab1

In [76]:
nih_awards_url = "https://report.nih.gov/award/index.cfm?ot=&fy=2024&state=&ic=&fm=&orgid=&distr=&rfa=&om=n&pid=#tab1"
response = requests.get(nih_awards_url)
soup = BeautifulSoup(response.content, "html.parser")

table = soup.find("table", {"id": "locationsortable"})

In [79]:
# Extract headers
headers = [th.text.strip() for th in table.find_all("th")]

In [81]:
# Extract rows
rows = []
for tr in table.find_all("tr"):
    row_data = [td.text.strip() for td in tr.find_all("td")]
    if row_data:  # Skip empty rows
        rows.append(row_data)

In [83]:
df_2024 = pd.DataFrame(rows, columns=headers)

In [85]:
df_2024.tail()

,Location,Awards,Funding
51,West Virginia,89,"$54,131,746"
52,Wisconsin,"1,045","$595,126,918"
53,Wyoming,20,"$11,686,417"
54,Foreign,631,"$265,899,217"
55,Total,"65,073","$37,265,104,195"


In [94]:
total_2024 = int(df_2024.iloc[55]["Funding"].replace(",", "").replace("$", ""))

In [109]:
ave_funding_5_years = (total_2024 + total_2020_23) / 5

In [110]:
print(f"The average funding for the last five years is: ${ave_funding_5_years:,.0f}")


The average funding for the last five years is: $42,952,415,839


In [114]:
# Calculate the sum of 'df_confirmed['award_remaining']', excluding null or NaN values
sum_award_remaining = df_confirmed['award_remaining'].dropna().sum()
print(f"The sum of 'award_remaining' is: ${sum_award_remaining:,.0f}")


The sum of 'award_remaining' is: $1,956,412,512


In [121]:
# sum_award_remaining as a percent of the ave_funding_5_years
proportion_remaining = (sum_award_remaining / ave_funding_5_years) * 100
print(f"Remaining funding as proportion of average funding last five years: {round(proportion_remaining, ndigits=2)}%")

Remaining funding as proportion of average funding last five years: 4.55%
